# American Census

In [2]:
import numpy as np
import pandas as pd

from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

## Global config

In [3]:
# Data config
path_to_csv = "adult.csv"


## Load data

In [4]:
X_train_raw = pd.read_csv("X_train.csv")
y_train_raw = pd.read_csv("y_train.csv")

X_test_raw = pd.read_csv("X_test.csv")


## Helpers

In [12]:
# TODO

## Data exploration

### Question 1

In [37]:
# TODO

### Question 2

In [38]:
# TODO

## Data exploration

### Question 1

In [5]:
# Cleanup data
from sklearn.impute import SimpleImputer
from math import isnan

imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
imp.fit(X_train_raw)
X_train_imputed = imp.transform(X_train_raw)

### Question 2

In [6]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
enc.fit(X_train_imputed)
X_train_enc = enc.transform(X_train_imputed)

### Question 3

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train_enc)
X_train_processed = scaler.transform(X_train_enc)

## Experiments

### Question 1

In [8]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

models = {
    "GaussianNB": GaussianNB(),
    "DecisionTreeClassifier": DecisionTreeClassifier(max_depth=None, min_samples_leaf=2, random_state=42),
    "KNeighborsClassifier": KNeighborsClassifier(n_neighbors=3, weights="distance"),
    "SVM": SVC(C=10, kernel="poly", random_state=42),
    "LogisticRegression": LogisticRegression(C=10, penalty="none", random_state=42),
}

# Split
X_train, X_test, y_train, y_test = train_test_split(X_train_processed, y_train_raw, test_size=0.1, random_state=42, shuffle=True, stratify=y_train_raw)

# TODO: k-fold cross validation?

### Question 2

In [13]:
for name, model in models.items():
    model.fit(X_train, y_train.values.ravel())
    prediction = model.predict(X_test)
    f1 = f1_score(prediction, y_test, average="weighted")
    acc = accuracy_score(prediction, y_test)
    print(f"{name}, f1: {f1}, acc: {acc}")

# TODO: plot metrics

GaussianNB, f1: 0.7734035994894339, acc: 0.7831695331695332
DecisionTreeClassifier, f1: 0.7947386575829911, acc: 0.7893120393120393
KNeighborsClassifier, f1: 0.7999294283680028, acc: 0.7997542997542998
SVM, f1: 0.8363039568914115, acc: 0.8212530712530712
LogisticRegression, f1: 0.8207583147810715, acc: 0.8034398034398035


/home/rkochar/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1504: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


GaussianNB  | mean accuracy:  0.7857857960612329  | std accuracy:  0.005061488204816713
DecisionTreeClassifier  | mean accuracy:  0.7863628154707551  | std accuracy:  0.014234527944814484
KNeighborsClassifier  | mean accuracy:  0.7951126747847944  | std accuracy:  0.008621848799248811
SVM  | mean accuracy:  0.8041703913495329  | std accuracy:  0.00781977614973563
LogisticRegression  | mean accuracy:  0.782245647539234  | std accuracy:  0.006742389227709247


### Question 3

In [18]:
# TODO: GridSearchCV

ValueError: could not convert string to float: ' Private'

### Question 4

In [ ]:
#TODO

### Question 5

In [10]:
#TODO
prediction = np.array([-1] * len(X_test)) #TODO replace this with you own prediction
pd.DataFrame(prediction).to_csv("GROUP_classes_problem_census.txt", index=False, header=False)